In [9]:
import math
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import *

In [10]:
class Dist:
    def __init__(self,num_res,max_nw_size,job_len):
        #type of resource used
        self.num_res = num_res#环境中任务的种类
        #the max size of resource of each job
        self.max_nw_size = max_nw_size#新任务需求的最大资源（每个任务需求的资源不能超过这个值）
        #the max time used of each job
        self.job_len = job_len#新任务最大的执行时间，新任务执行时间不能超过这个值
        
        #the chance of create a small job
        self.job_small_chance = 0.8#小任务的概率是0.8
        
        #the band of big job lenth
        self.job_len_big_lower = job_len * 2 / 3#一个大任务执行时间的下界是job_len的2/3
        self.job_len_big_upper = job_len#一个大任务执行时间的上界是job_len
        
        #the band of small job lenth
        self.job_len_small_lower = 1#一个小任务执行时间的下界是1
        self.job_len_small_upper = job_len / 5#一个小任务执行时间的上界是len_job的1/5
        
        #the dominant resource size 
        self.dominant_res_lower = max_nw_size / 2#一个任务占主导地位的需求资源的的下界是max_nx_size的1/2
        self.dominant_res_upper = max_nw_size#一个任务占主导地位的需求资源的的上界是max_nx_size
        
        #the other resource size
        self.other_res_lower = 1#一个任务占非主导地位的需求资源的的下界是1
        self.other_res_upper = max_nw_size / 5#一个任务占非主导地位的需求资源的的上界是max_nx_size的1/5
    
    def normal_dist(self):#完全随机地生成一个job
        # new work duration
        nw_len = np.random.randint(1, self.job_len + 1)  # same length in every dimension

        nw_size = np.zeros(self.num_res)

        for i in range(self.num_res):
            nw_size[i] = np.random.randint(1, self.max_nw_size + 1)

        return nw_len, nw_size
    
    def bi_model_dist(self):#这是用来生成一个job的方法

        # -- job length --
        if np.random.rand() < self.job_small_chance:  # small job，生成一个随机数，如果小于0.8，则生成一个小任务
            nw_len = np.random.randint(self.job_len_small_lower,
                                           self.job_len_small_upper + 1)#随机生成小任务的执行时间
        else:  # big job
            nw_len = np.random.randint(self.job_len_big_lower,
                                           self.job_len_big_upper + 1)#随机生成一个大任务的执行时间



        # -- job resource request --
        nw_size = np.zeros(self.num_res)#生成一个数组，用来记录一个job的两种资源的需求情况
        dominant_res = np.random.randint(0, self.num_res)#生成一个0 或者 1 的数，用来表示这个任务占主导地位的资源是第一种或者第二种资源
        for i in range(self.num_res):#给这个任务随机生成两种资源的需求情况
            if i == dominant_res:
                nw_size[i] = np.random.randint(self.dominant_res_lower,
                                                   self.dominant_res_upper + 1)
            else:
                nw_size[i] = np.random.randint(self.other_res_lower,
                                                   self.other_res_upper + 1)

        return nw_len, nw_size#返回值是一个数组，第一个数是这个任务的执行时间，第二个数是一个数组，代表两种资源的需求



In [11]:
num_res = 2#环境中任务的种类
max_nw_size =10#新任务需求的最大资源（每个任务需求的资源不能超过这个值）
max_job_len = 15#新任务最大的执行时间，新任务执行时间不能超过这个值
simu_len = 50
new_job_rate=0.3
T_period=max_job_len*simu_len
res_slot = 10

In [12]:
dist=Dist(num_res,max_nw_size,max_job_len)

In [13]:
nw_len,nw_res=dist.bi_model_dist()

In [14]:
nw_len_lst

array([ 0,  1,  0,  0,  0,  1,  2,  0,  0,  0,  3,  0,  0,  2,  0,  0,  0,
        0,  0,  3,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, 12,  0,  0,  0,  0,  0,  3])

In [15]:
nw_len_lst=np.zeros(simu_len, dtype=int)
nw_res_lst=np.zeros((simu_len,num_res), dtype=int)
for i in range(simu_len):
    if np.random.rand() < new_job_rate:  # a new job comes，在每个工作的位置上随机生成一个数字，如果小于新工作率，则在当前位置新生成一个任务
        dist=Dist(num_res,max_nw_size,max_job_len)
        nw_len_lst[i], nw_res_lst[i, :] = dist.bi_model_dist()

In [16]:
T=[]
for i in range(T_period):
    T.append(i)

In [17]:
arrval_time_lst=[]
for i in range(simu_len):
    arrval_time_lst.append(0)
        
        

In [18]:
jobs=[]
for i in range(simu_len):
    jobs.append(str(i+1))

In [19]:
dauer_dict={}
i=0
for job in jobs:
    dauer_dict[job]=nw_len_lst[i]
    i=i+1

In [20]:
arrval_dict={}
i=0
for job in jobs:
    arrval_dict[job]=arrval_time_lst[i]
    i+=1

In [21]:
res_dict={}
i=0
for job in jobs:
    res_dict[job]=nw_res_lst[i]
    i+=1

In [22]:
workload = np.zeros(num_res)
for i in range(num_res):
    workload[i] = \
    np.sum(nw_res_lst[:, i] * nw_len_lst) / \
    float(res_slot) / \
    float(len(nw_len_lst))
    print("Load on # " + str(i) + " resource dimension is " + str(workload[i]))

Load on # 0 resource dimension is 0.462
Load on # 1 resource dimension is 0.544


In [23]:
m=Model()

Using license file /Users/11yunyi/gurobi.lic
Academic license - for non-commercial use only


In [24]:
x=m.addVars(jobs,T,name='start_time_bool',vtype=GRB.BINARY)
omiga=m.addVars(jobs,lb=0,ub=T_period,name='end_time',vtype=GRB.INTEGER)
alpha=m.addVars(jobs,lb=0,ub=T_period,name='start_time',vtype=GRB.INTEGER)
y=m.addVars(jobs,T,name='dauer_time_bool',vtype=GRB.BINARY)
slowdown=m.addVars(jobs,lb=0,name='slow_down',vtype=GRB.INTEGER)

In [25]:
for job in jobs:
    if dauer_dict[job]==0:
        m.addConstr(slowdown[job]==1)
    else:
        m.addConstr(slowdown[job]==(omiga[job]-arrval_dict[job])/dauer_dict[job])


In [26]:
m.addConstrs((alpha[job]==omiga[job]-dauer_dict[job] for job in jobs))

{'1': <gurobi.Constr *Awaiting Model Update*>,
 '2': <gurobi.Constr *Awaiting Model Update*>,
 '3': <gurobi.Constr *Awaiting Model Update*>,
 '4': <gurobi.Constr *Awaiting Model Update*>,
 '5': <gurobi.Constr *Awaiting Model Update*>,
 '6': <gurobi.Constr *Awaiting Model Update*>,
 '7': <gurobi.Constr *Awaiting Model Update*>,
 '8': <gurobi.Constr *Awaiting Model Update*>,
 '9': <gurobi.Constr *Awaiting Model Update*>,
 '10': <gurobi.Constr *Awaiting Model Update*>,
 '11': <gurobi.Constr *Awaiting Model Update*>,
 '12': <gurobi.Constr *Awaiting Model Update*>,
 '13': <gurobi.Constr *Awaiting Model Update*>,
 '14': <gurobi.Constr *Awaiting Model Update*>,
 '15': <gurobi.Constr *Awaiting Model Update*>,
 '16': <gurobi.Constr *Awaiting Model Update*>,
 '17': <gurobi.Constr *Awaiting Model Update*>,
 '18': <gurobi.Constr *Awaiting Model Update*>,
 '19': <gurobi.Constr *Awaiting Model Update*>,
 '20': <gurobi.Constr *Awaiting Model Update*>,
 '21': <gurobi.Constr *Awaiting Model Update*>,
 

In [27]:
for i in range(simu_len):
    m.addConstrs(alpha[job]>=arrval_dict[job] for job in jobs)

In [28]:
for job in jobs:
    m.addConstr(quicksum(x[(job),time] for time in T)==1)

In [29]:
for job in jobs:
    m.addConstr(quicksum(y[(job),time] for time in T)==dauer_dict[job])

In [30]:
for job in jobs:
    for timee in T[:T_period-max(dauer_dict.values())]:
        m.addGenConstrIndicator(x[job,timee],True,quicksum(y[job,timee+timeee] for timeee in range(dauer_dict[job])),GRB.EQUAL,dauer_dict[job])

In [31]:
for i in range(len(T)):
    for j in range(num_res):
        m.addConstr(quicksum(y[(job),T[i]]*res_dict[job][j] for job in jobs)<=res_slot)

In [32]:
for job in jobs:
    m.addConstr(quicksum(x[(job),time]*time for time in T)==alpha[job])  

In [33]:
m.setObjective(1/simu_len*quicksum(slowdown[job] for job in jobs),GRB.MINIMIZE)

In [34]:
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4250 rows, 75150 columns and 140667 nonzeros
Model fingerprint: 0xf19c7868
Model has 36850 general constraints
Variable types: 0 continuous, 75150 integer (75000 binary)
Coefficient statistics:
  Matrix range     [8e-02, 7e+02]
  Objective range  [2e-02, 2e-02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 1e+01]
Presolve added 4749 rows and 0 columns
Presolve removed 0 rows and 54808 columns
Presolve time: 2.90s
Presolved: 8999 rows, 20342 columns, 73391 nonzeros
Variable types: 0 continuous, 20342 integer (15166 binary)
Found heuristic solution: objective 2.6000000

Root relaxation: objective 1.542681e+00, 1211 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.54268    0   70    2.60000    1.54268  40.7%     -    3s
H    0     0                       

In [37]:
m.ObjVal

1.8200000000000007